In [83]:
# grant Cloud SQL Client role to authenticated user
!gcloud auth application-default login

current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

print(current_user)

!gcloud projects add-iam-policy-binding utopian-splicer-400911 --member=user:{current_user[0]} --role="roles/cloudsql.client"

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=P17ZdBGlasyqMGMi6IXqDPtHIpbNKI&access_type=offline&code_challenge=NAv54dHu2ohfm9skwM3tXGV9d4bAF6zrj1Y_7nqCBmM&code_challenge_method=S256


Credentials saved to file: [C:\Users\Lenovo\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "utopian-splicer-400911" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


['ec23661@qmul.ac.uk']
bindings:
- members:
  - serviceAccount:service-391234968399@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:391234968399@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-391234968399@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-391234968399@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - user:ec23661@qmul.ac.uk
  role: roles/cloudsql.client
- members:
  - serviceAccount:service-391234968399@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-391234968399@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-391234968399@containerregistry.iam.gserviceaccount.com
  role: roles/containerr

Updated IAM policy for project [utopian-splicer-400911].


In [84]:
# !gcloud projects add-iam-policy-binding YOUR_PROJECT_ID --member "serviceAccount:NAME@YOUR_PROJECT_ID.iam.gserviceaccount.com" --role "roles/owner"

In [85]:
!gcloud services enable sqladmin.googleapis.com

In [86]:
region = "europe-west2" 
instance_name = "kienps" 

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for database 'root' user: ")
  !gcloud sql instances create {instance_name} --database-version=MYSQL_8_0 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql_iam_authentication=On

Found existing MySQL Cloud SQL Instance!


In [95]:
!gcloud sql databases create cloud_project_4 --instance={instance_name}

instance: kienps
name: cloud_project_4
project: utopian-splicer-400911


Creating Cloud SQL database...
.done.
Created database [cloud_project_4].


In [96]:
!gcloud sql users create user --instance={instance_name} --password="test"

Creating Cloud SQL user...
.done.
Created user [user].


In [97]:
# initialize parameters
project_id='utopian-splicer-400911'
region = 'europe-west2'
instance_name = 'kienps'

INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "user"
DB_PASS = "test"
DB_NAME = "cloud_project_4"

Your instance connection name is: utopian-splicer-400911:europe-west2:kienps


In [98]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

In [99]:
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  db_conn.execute(
    sqlalchemy.text(
      "CREATE TABLE IF NOT EXISTS user "
      "( id SERIAL NOT NULL, username VARCHAR(255) NOT NULL, "
      "email VARCHAR(255) NOT NULL, password VARCHAR(1000) NOT NULL, "
      "PRIMARY KEY (id));"
    )
  )

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # insert data into our ratings table
  insert_stmt = sqlalchemy.text(
      "INSERT INTO user (email, password, username) VALUES (:email, :password, :username)",
  )

  # insert entries into table
  db_conn.execute(insert_stmt, parameters={ "email": "ok@gmail.com","password": "testing1", "username": "kienps"})
  db_conn.execute(insert_stmt, parameters={ "email": "ok1@gmail.com", "password": "testing2", "username": "Diba"})
  db_conn.execute(insert_stmt, parameters={"email": "ok2@gmail.com", "password": "testing3", "username": "Admin"})

  # commit transactions
  db_conn.commit()

  # query and fetch ratings table
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM user")).fetchall()

  # show results
  for row in results:
    print(row)

(1, 'kienps', 'ok@gmail.com', 'testing1')
(2, 'Diba', 'ok1@gmail.com', 'testing2')
(3, 'Admin', 'ok2@gmail.com', 'testing3')
